### Counters on entrancies data (csv)

In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
user_preferences = np.array([0, 3, 2, 0, 1, 2, 3, 0])  # button presses

In [4]:
df = pd.read_excel('../../data.xlsx', nrows=100)
df

,CounterReadingID,CounterID_ASTA,SequenceNumber,StartTime,EndTime,Visits,ASTA_Counters.CounterID_PAVE,ASTA_Counters.Name_ASTA,ASTA_Counters.InstallationDate,ASTA_Counters.NationalParkCode,ASTA_Counters.Municipality,ASTA_Counters.RegionalUnit,ASTA_Counters.RegionalEntity,PAVE_Counters.Globalid,PAVE_Counters.Name,PAVE_Counters.Manager,PAVE_Counters.AdditionalInfo,PAVE_Counters.CoordinateNorth,PAVE_Counters.CoordinateEast
0,263345,1099,4,2013-07-19 15:00:00,2013-07-19 16:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
1,263349,1099,8,2013-07-19 19:00:00,2013-07-19 20:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
2,263350,1099,9,2013-07-19 20:00:00,2013-07-19 21:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
3,263351,1099,10,2013-07-19 21:00:00,2013-07-19 22:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
4,263352,1099,11,2013-07-19 22:00:00,2013-07-19 23:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
5,263353,1099,12,2013-07-19 23:00:00,2013-07-20 00:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
6,263354,1099,13,2013-07-20 00:00:00,2013-07-20 01:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
7,263355,1099,14,2013-07-20 01:00:00,2013-07-20 02:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
8,263356,1099,15,2013-07-20 02:00:00,2013-07-20 03:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166
9,263357,1099,16,2013-07-20 03:00:00,2013-07-20 04:00:00,0,190927,Varkaankuru 2013,2013-07-12,34361,273,800,800022,{47664E5F-0E50-4F64-B000-77D8A61BC5BA},Röhkömukka Varkaankurulle kävijälaskuri (kesä),J. Ylläsjärvi,EcoCounter,7500782,381166


For further analysis see [tcdf model notebook](ml/tcdf_notebook.ipynb).

### Weather API

https://rapidapi.com

In [7]:
import requests
import credentials
import re

cities = ["London,uk", "Porto,pt", "Paris,fr"]
weather_dict = {}

def city_forecast(city):
  response = requests.get(
          "https://community-open-weather-map.p.rapidapi.com/forecast?q="+city,
          headers={
          "X-RapidAPI-Host": "community-open-weather-map.p.rapidapi.com",
          "X-RapidAPI-Key": credentials.rapidapi_key
        },
  )

  return response.json()

for city in cities:
  weather_dict[city] = city_forecast(city)

In [8]:
def get_day_weather(pred):

  pattern = re.compile("s(?P<hour>dd):dd:dd")
  t = pattern.search(pred['dt_txt'])

  if int(t.group('hour')) >= 10 and int(t.group('hour')) <= 19:    
        return True

  return False

day_weather = {}

for city in weather_dict.keys():
  day_weather[city] = list(filter(get_day_weather, weather_dict[city]['list']))

In [ ]:
def travel_estimator(weather_stat):
  estimation = {}
  for city in weather_stat.keys():    
        estimation[city] = {}
        estimation[city]['clear_sky_count'] = 0
        estimation[city]['av_temp'] = 0
        for prediction in day_weather[city]:
          estimation[city]['av_temp'] += prediction['main']['temp']
          if prediction['weather'][0]['description'] in ['clear sky', 'few clouds']:
            estimation[city]['clear_sky_count'] += 1          
        estimation[city]['av_temp'] = round(estimation[city]['av_temp'] / len(day_weather[city]), 2)
        # convert temperature to celsius
        estimation[city]['av_temp_cels'] = round(estimation[city]['av_temp'] - 273.15, 2)
  return estimation

travel_rating = travel_estimator(day_weather)

# sort cities by clear sky forecasts
sorted_travel_rating = sorted(travel_rating.items() ,  key=lambda x: x[1]['av_temp_cels'] )

for city in sorted_travel_rating:
  print(city[0])
  print('Clear sky forecasts:', city[1]['clear_sky_count'])  
  print('Average temperature:', city[1]['av_temp_cels'], "C")
  print('n')